## Lab 10 — Superdense Coding

Superdense coding is a quantum communication protocol where Alice and Bob begin with a shared entangled pair, each holding one qubit. By applying one of four possible transformations to her qubit, Alice can encode two classical bits of information into the joint state of the pair. She then sends her qubit to Bob, who, by performing a joint measurement on both qubits in the Bell basis, can determine exactly which transformation Alice applied, thereby retrieving the two encoded bits. This process allows the transmission of two classical bits by sending only one qubit, provided that prior entanglement is available.

### Task

Alice intends to send an **8-bit** random key to Bob using the superdense coding protocol. The task is to create a circuit that tests this protocol.

### Expected Output

The program should report whether the transmission was **successful** or **failed**, and display both the key Alice sent and the key Bob received.

### Optional Challenge

Modify the code so that it uses one circuit with 8 qubit lines to transfer the entire 8-bit key in parallel.


In [ ]:
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
import random

# --- Helper functions for superdense coding ---
def bell_pair(qc, a, b):
    """Create a Bell pair between qubits a (Alice) and b (Bob)."""
    qc.h(a)
    qc.cx(a, b)

def encode(qc, a, b0, b1):
    """Alice encodes her two classical bits onto her qubit."""
    if (b0, b1) == (0, 1):
        qc.x(a)
    elif (b0, b1) == (1, 0):
        qc.z(a)
    elif (b0, b1) == (1, 1):
        qc.x(a)
        qc.z(a)

def decode(qc, a, b):
    """Bob decodes to recover Alice's two classical bits."""
    qc.cx(a, b)
    qc.h(a)

# ------------------------------------------------
#                main program
# ------------------------------------------------

# --- Alice creates an 8-bit random key ---
alice_bits = [random.randint(0, 1) for _ in range(8)]
print("Alice's key:    ", "".join(map(str, alice_bits)))

# --- Split key into pairs for superdense coding ---
bit_pairs = []
for i in range(0, len(alice_bits), 2):
    first_bit = alice_bits[i]
    second_bit = alice_bits[i + 1]
    bit_pairs.append((first_bit, second_bit))

# --- Simulate sending the key ---
simulator = AerSimulator()
bob_bits = []

for b0, b1 in bit_pairs:
    qc = QuantumCircuit(2, 2)  # 2 qubits (Alice, Bob), 2 classical bits
    
    # Share entanglement
    bell_pair(qc, 0, 1)
    
    # Alice encodes her 2 bits
    encode(qc, 0, b0, b1)
    
    # Bob decodes
    decode(qc, 0, 1)
    
    # Measure both qubits
    qc.measure([0, 1], [0, 1])
    
    # Run the circuit
    result = simulator.run(qc, shots=1, memory=True).result()
    mem = result.get_memory()[0]  # e.g., '10' (c1c0)
    
    # Extract bits in correct order (first_bit = c0, second_bit = c1)
    c1, c0 = int(mem[0]), int(mem[1])
    bob_bits.extend([c0, c1])


# --- Show results ---
print("Bob's key:      ", "".join(map(str, bob_bits)))

if bob_bits == alice_bits:
    print("Transmission OK – keys match.")
else:
    print("Transmission FAILED – keys differ.")
